<a href="https://colab.research.google.com/github/mammubarak/3D-Printer-Material-Volume-Calculator-.NET-Web-services/blob/main/IBM_ART_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install adversarial-robustness-toolbox
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.4/870.4 kB 49.3 MB/s eta 0:00:00


In [5]:
from ultralytics import YOLO

# Load your pre-trained YOLOv10 model
model_path = "/content/best.pt"  # Replace with the correct path
model = YOLO(model_path)


In [8]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="cehidgzm3qU3AW20KpIk")
project = rf.workspace("moin").project("car_license_plates")
version = project.version(1)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Car_License_Plates-1 in yolov8:: 100%|██████████| 1402/1402 [00:00<00:00, 7363.28it/s]


In [9]:
from art.estimators.object_detection import PyTorchObjectDetector
import torch

# Create a custom forward function
class YOLOv10Wrapper(torch.nn.Module):
    def __init__(self, model):
        super(YOLOv10Wrapper, self).__init__()
        self.model = model

    def forward(self, x):
        # Your custom logic to run the YOLO model
        results = self.model.predict(x)
        return results

# Wrap your YOLO model
yolo_wrapper = YOLOv10Wrapper(model)

# Initialize the ART ObjectDetector with your wrapped model
art_model = PyTorchObjectDetector(
    model=yolo_wrapper,
    input_shape=(3, 640, 640),  # Modify based on your input dimensions
    clip_values=(0, 1),  # Clip values based on your input preprocessing
    attack_losses="yolo"  # Use appropriate loss for YOLO if needed
)


engine/trainer: task=detect, mode=train, model=/content/best.pt, data=/content/data3.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=F

100%|██████████| 755k/755k [00:00<00:00, 122MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1      9856  ultralytics.nn.modules.block.SCDown          [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1     36096  ultralytics.nn.modules.block.SCDown          [128

100%|██████████| 5.35M/5.35M [00:00<00:00, 280MB/s]


AMP: checks passed ✅


...

    100/100         3G     0.6367     0.3598      1.589          9        640: 100%|██████████| 38/38 [00:14<00:00,  2.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.23it/s]

                   all         64         68      0.884      0.735      0.833       0.51



100 epochs completed in 0.449 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 5.8MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 5.8MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.14 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10n summary (fused): 285 layers, 2,694,806 parameters, 0 gradients, 8.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.54it/s]


                   all         64         68      0.907      0.717      0.835      0.515
Speed: 0.3ms preprocess, 3.1ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs/detect/train2


In [16]:
import shutil
import os

# Specify the folder you want to download
folder_name = 'runs'
output_zip = f'{folder_name}.zip'

# Compress the folder into a zip file
shutil.make_archive(folder_name, 'zip', folder_name)

# Download the zip file
from google.colab import files
files.download(output_zip)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
%pip show art



NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [34]:
import numpy as np
from art.estimators.estimator import BaseEstimator
from art.estimators.LossGradientsMixin import LossGradientsMixin
import torch

class CustomYOLOEstimator(BaseEstimator, LossGradientsMixin):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.input_shape = (3, 640, 640)  # Update this based on your model's input shape

    def predict(self, x):
        # Convert input to the required format and run the model
        with torch.no_grad():
            x_tensor = torch.from_numpy(x).float().permute(0, 3, 1, 2)  # HWC to CHW
            predictions = self.model(x_tensor)
            # Process predictions as necessary (e.g., get classes, bounding boxes)
            return predictions  # Return in the expected format

    def loss_gradients(self, x, y):
        # Implement loss gradient calculation
        # This would typically involve computing gradients for the model's loss function
        # You'll need to define a loss function based on your use case (e.g., classification loss)
        # Return gradients
        pass

    def get_logits(self, x):
        """Get logits from the model (if applicable)."""
        with torch.no_grad():
            return self.model(torch.from_numpy(x).float()).cpu().numpy()


ModuleNotFoundError: No module named 'art.estimators.LossGradientsMixin'

In [ ]:
x_test_adv = attack.generate(x=x_test)

# Evaluate the model on adversarial examples
predictions = art_model.predict(x_test_adv)


In [ ]:
import torch

# Save the model weights
model_save_path = "adversarially_trained_yolov10.pt"
torch.save(yolo_wrapper.state_dict(), model_save_path)